In [ ]:
import warnings

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import roc_auc_score, accuracy_score

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc

## Obtén los datos

In [ ]:
data = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
data.head()

In [ ]:
# drop_columns = ['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours']
# data = data.drop(drop_columns, axis=1)


### Leer los datos separados


Analizar estadísticamente el dataset: Separar el an análisis de las variables numéricas y categóricas.


In [ ]:
# Seleccionar las columnas numericas
lista_num = data._get_numeric_data().columns
# Crear DF con las columnas numericas
data_num = data[lista_num]
# Crear DF con las columnas categoricas.
data_cat = data.drop(columns=data_num)

In [ ]:
# X=data_cat.drop(columns=['Attrition'])
# y=data_cat['Attrition']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Identificar las columnas categóricas
categorical_columns = [
    'BusinessTravel', 'Department', 'EducationField', 'Gender',
    'JobRole', 'MaritalStatus', 'OverTime',"Attrition"
]
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_data = encoder.fit_transform(data[categorical_columns])

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

In [ ]:
data_encoded = pd.concat([data_num, encoded_df], axis = 1)
data_encoded

Renombrar las columnas

In [ ]:
data_encoded = data_encoded.rename(columns={
    'OverTime_Yes': 'OverTime',
    'Attrition_Yes': 'Attrition'
})

 # Preseleccionar cuales son las variables más importantes del dataset
- Attrition
- OverTime
- MaritalStatus_Single
- JobRole_Sales Representative
- BusinessTravel_Travel_Frequently
- JobRole_Laboratory Technician
- Department_Sales
- DistanceFromHome
- EducationField_Technical Degree
- EducationField_Marketing


In [ ]:
abs(data_encoded.corr()["Attrition"].sort_values(ascending=False).head(11))

# Afinar los Modelos

In [ ]:
X = data_encoded.drop(columns=['Attrition'])
y = data_encoded['Attrition']


## Balancear Clases

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(X, y)
print("Before Smoote" , y.value_counts())
print()
print("After Smoote" , y_smote.value_counts())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_smote, y_smote, test_size=0.2, random_state=42)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Modelos (Sin Hiperparametrizar):

- WeightedEnsemble_L2 - RegLog - Accuracy: 90.48%[[252   3][ 25  14]]
- ___________  Regresion Logistica - Accuracy: 89.12% - [[244  11][ 21  18]]
- _______________Random Forest - Accuracy: 87.41% - [[254   1][ 36   3]]
========================================================================================
- ____________________CatBoost - Accuracy: 87.41% - [[252   3][ 34   5]]
- ____________________ XGBoost - Accuracy: 87.07% - [[246   9][ 29  10]]


Modelos Hiperparametrizados:
- WeightedEnsemble_L2 - RegLog -
- ___________  Regresion Logistica -
- _______________Random Forest -

Modelos Suavizados (Hiperparametrizados):

- WeightedEnsemble_L2 - RegLog - Accuracy: 87.07% [[249   6][ 32   7]]
- ___________  Regresion Logistica - Accuracy: 89.80%[[249   6] [ 24  15]]
- _______________Random Forest - Accuracy: 86.73% [[251   4] [ 35   4]]

### Random Forest Hiperparametrizado

In [ ]:
RandomForestClassifier().get_params()

In [ ]:
rf = RandomForestClassifier(random_state=42)

In [ ]:
# Definir los hiperparámetros para la búsqueda
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 5}, 'balanced']
}

# Realizar la búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [ ]:
# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Evaluar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_
yhat = best_model.predict(X_test)

In [ ]:
cfm = confusion_matrix(y_test, yhat) # ()
disp = ConfusionMatrixDisplay(cfm)
disp.plot()
plt.show()
print(cfm) # Pinto la matriz de confusion.

In [ ]:
accuracy = accuracy_score(y_test, yhat)
conf_matrix = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

#### WeightedEnsemble_L2 - RegLog - Accuracy: 89.80%[[250   5][ 25  14]]

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


In [ ]:

# Crear modelos base
base_estimators = [
    ('lr', LogisticRegression(max_iter=1000, random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]

# Crear el metamodelo
meta_estimator = LogisticRegression(random_state=42)

# Crear el modelo de stacking
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_estimator, cv=5)

# Definir el espacio de búsqueda de hiperparámetros
param_grid = {
    'lr__C': [0.1, 1, 10],
    'dt__max_depth': [None, 10, 20, 30],
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
    'gb__n_estimators': [50, 100, 200],
    'gb__learning_rate': [0.01, 0.1, 1],
    'final_estimator__C': [0.1, 1, 10]
}

# Crear el GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=stacking_model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el modelo de stacking con búsqueda en cuadrícula
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_stacking_model = grid_search.best_estimator_

# Realizar predicciones sobre el conjunto de prueba
yhat = best_stacking_model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, yhat)
print(f'Precisión del mejor modelo de stacking: {accuracy:.2f}')

In [ ]:
accuracy = accuracy_score(y_test, yhat)
conf_matrix = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

In [ ]:
# Predicciones en datos de entrenamiento con el mejor modelo
train_predictions_best = best_stacking_model.predict(X_train)
train_accuracy_best = accuracy_score(y_train, train_predictions_best)

# Precisión en datos de prueba (ya calculada anteriormente)
test_accuracy_best = accuracy_score(y_test, yhat)

print(f"Train Accuracy (Stacking Model): {train_accuracy_best:.2f}")
print(f"Test Accuracy (Stacking Model): {test_accuracy_best:.2f}")



### Regresion Logistica Escalada Hiperparametrizada.


In [ ]:
LogisticRegression().get_params()

In [ ]:
#Escalamos primero
objeto_escalador = StandardScaler() # creo el objeto escalador, escalara el dataframe
objeto_escalador.fit(X_train) # le enseñamos la estructura de columnas(33 columnas) y la media y varianza de las 33 columnasfrom sklearn.preprocessing import StandardScaler

# ======================================

objeto_escalador.transform(X_train)

# ======================================

X_train[:] = objeto_escalador.transform(X_train)
X_test[:] = objeto_escalador.transform(X_test)
# sobreescrivo los valores
# X_train = ... esto sobreescrivira el objeto X_train
# el escalador se hace fit con train y se aplica a train y test

In [ ]:
clf = LogisticRegression() # ', class_weight='balanced'class_weight da mas peso a los 1.

# Definir los parámetros para la búsqueda en cuadrícula

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l2'],  # Usamos solo 'l2' para simplificar
    'solver': ['liblinear', 'lbfgs', 'saga'],
    'tol': [1e-4, 1e-5],
    'max_iter': [200, 300, 500]
}


# Crear el GridSearchCV
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el GridSearchCV
grid_search.fit(X_train, y_train)



In [ ]:

# Obtener y mostrar los mejores parámetros
best_params = grid_search.best_params_
print("Mejores parámetros encontrados:")
print(best_params)

# Evaluar el mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
yhat = best_model.predict(X_test)
accuracy = accuracy_score(y_test, yhat)
print(f'Precisión del mejor modelo de regresión logística: {accuracy}')

In [ ]:
cfm = confusion_matrix(y_test, yhat) # ()
disp = ConfusionMatrixDisplay(cfm)
disp.plot()
plt.show()
print(cfm) # Pinto la matriz de confusion.

In [ ]:
accuracy = accuracy_score(y_test, yhat)
conf_matrix = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

In [ ]:
    #! Hay Overfitting?

# Ajustar el modelo en los datos de entrenamiento
best_model.fit(X_train, y_train)
# Predicciones en datos de entrenamiento
train_predictions = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
# Predicciones en datos de prueba
test_predictions = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


### Random Forest Hiperparametrizado

In [ ]:
RandomForestClassifier().get_params()

In [ ]:
rf = RandomForestClassifier(random_state=42)
# Definir los hiperparámetros para la búsqueda
param_grid = {
    'n_estimators': [400, 700, 800],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [None, 'balanced', {0: 1, 1: 10}]
}
# Realizar la búsqueda de hiperparámetros con GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Evaluar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_
yhat = best_model.predict(X_test)

print(f"\nPrecisión del modelo: {accuracy:.4f}")


In [ ]:
# - _______________Random Forest - Accuracy: 87.76% - [[253   2][ 33  5]]

In [ ]:
cfm = confusion_matrix(y_test, yhat) # ()
disp = ConfusionMatrixDisplay(cfm)
disp.plot()
plt.show()
print(cfm) # Pinto la matriz de confusion.

In [ ]:
accuracy = accuracy_score(y_test, yhat)
conf_matrix = confusion_matrix(y_test, yhat)
class_report = classification_report(y_test, yhat)
print(f'Accuracy: {accuracy * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)